#### REGULARIZATION

We want to create two prediction models both for solar energy output and wind energy output. In order to predict these outputs we want to build a model that selects the best possible predictors maximizing the feature significance, avoiding colinearity, ensuring sparisty, considering the best feature transformations... To do so we will apply different models like Lasso, and Ridge and then we can apply Holistic Regression to see how the model is improved by it. We will run all of these models by all predictors of solar and energy output. 

In [16]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames
#read train and validation data and split it into X and y
X_train_solar=Matrix(DataFrame(CSV.File("X_train_solar.csv")))
y_train_solar=Matrix(DataFrame(CSV.File("y_train_solar.csv")))
X_train_wind=Matrix(DataFrame(CSV.File("X_train_wind.csv")))
y_train_wind=Matrix(DataFrame(CSV.File("y_train_wind.csv")))

X_valid_solar=Matrix(DataFrame(CSV.File("X_valid_solar.csv")))
y_valid_solar=Matrix(DataFrame(CSV.File("y_valid_solar.csv")))
X_valid_wind=Matrix(DataFrame(CSV.File("X_valid_wind.csv")))
y_valid_wind=Matrix(DataFrame(CSV.File("y_valid_wind.csv")));

### Functions

In [3]:
function lasso(X,y,lambda=0.25)
    
    #Build model
    model = Model(Gurobi.Optimizer)#we have defined the model, pass Gurobi optimizer into the model
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    n, p = size(X)
    
    @variable(model, beta[j=1:p])
    @variable(model, beta_abs[j=1:p])
    
    
    #Insert constraints

    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
    
    #sum((y-X*beta).^2)) = sum((y[i] - sum(X[i,j]*beta[j]) for j=1:p)^2 for i=1:n)
    
    #Insert objective
    @objective(model, Min, sum((y-X*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

lasso (generic function with 2 methods)

In [4]:
function mse(X,y, beta)
    return sum((y-X*beta).^2)/length(y)
end

mse (generic function with 1 method)

In [5]:
function correlation(X,p)
    correlation_matrix = Statistics.cor(X) #compute corr matrix
    n,m = size(correlation_matrix) #set sizes
    correlated_pairs=[] #empty list to store correlated pairs
    for i=1:n #for all features
        for j=i+1:m 
            if abs(correlation_matrix[i,j])>p #if the abs value of the corr is higher than p
                push!(correlated_pairs, (i,j)) #append pair to list of correlated pairs
            end
        end
    end
    return correlated_pairs
end

correlation (generic function with 1 method)

In [6]:
#Build transformation function
function transformation(X)
    X_old=DataFrame(X, :auto) #define X as a df
    X_new=DataFrame() #new empty df 
    n,p=size(X_old)
    e=1
    for i=1:p #for each feature in X add 4 transformations
        X_new[!, "X$i"]=X_old[:,i] #transformation 1
        X_new[!, "Sqrt$i"]=X_old[:,i].^2 #transformation 2
        X_new[!, "Abs$i"]=sqrt.(abs.(X_old[:,i])) #transformation 3
        X_new[!, "Log$i"]=log.(abs.(X_old[:,i]).+e) #transformation 4
    end
    return(X_new) #we return a new df with all transformations, it will have size nxp*4
end

transformation (generic function with 1 method)

In [50]:
function holistic_regression(X,y,lambda=0.25, per=0.6, M=50, k=10) #add parameter per
    
    #Call functions
    X_new=Matrix(transformation(X)) #call function with all transformations of X
    HC = correlation(X_new, per) #call correlation function to compute hc_pairs
    
    #Set sizes
    n,p_new=size(X_new)
    
    #Build model
    model = Model(Gurobi.Optimizer)
    #model = Model(Gurobi.Optimizer, NonConvex = 2)#we have defined the model, pass Gurobi optimizer into the model
    #model = Model(with_optimizer(Gurobi.Optimizer, NonConvex = 2))
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    @variable(model, beta[1:p_new])
    @variable(model, beta_abs[1:p_new])
    @variable(model, z[1:p_new], Bin) #we add a binary variable
    

    #Insert constraints
    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
        
    #sparsity constraint: over all 60 features (including transformations) 
    @constraint(model, -M*z .<= beta)
    @constraint(model, beta .<= M*z)
    @constraint(model, sum(z) <= k) 
    
    #constraint on Transformation: from the 4 transformations per each feature we only select one
    for i=1:4:p_new
        #for j=i:i+3 #for every 4 transformations
        @constraint(model, sum(z[i:i+3])<=1)
        #end
    end #we get a vector with 15 features
    
    #constraint on HC pairs once we have selected 15 features 
    for (i,j) in HC
        @constraint(model, z[i] + z[j] <= 1)
    end #we can only take one of the pairs of correlated pairs
    #@constraint(model, sum(z[i])<=k) #ensure that the model has at most 8 features
    
        
    #Insert objective
    @objective(model, Min, sum((y-X_new*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

holistic_regression (generic function with 5 methods)

### SOLAR ENERGY OUTPUT

In [18]:
beta_lasso=lasso(X_train_solar,y_train_solar,0.25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


26-element Vector{Float64}:
     4.2093647114128725
    -1.5309979769072977
    -0.8824028062001327
     1.9116802490399556
   216.52631332891622
   -12.49098360321547
     7.631817286371418
    10.277016991626004
     0.5859704225440101
     0.11911001017780573
     3.824568986827269
     0.10472846328274894
    -3.2765141766725527
     0.42268676075881506
     5.486719230074171
   -40.48963755673621
     3.226686082766982
    -2.064111695818705
   -13.16697989755074
    53.64545205181411
   -10.82816662953645
     1.705722438924721e-6
 -1223.350040953427
     9.800267876089153e-11
     1.7057225056152977e-6
   -10.82816662953645

In [20]:
mse(X_valid_solar, y_valid_solar, beta_lasso)

87533.0624297519

In [26]:
correlation(X_train_solar,0.6) #correlated features

13-element Vector{Any}:
 (1, 2)
 (1, 3)
 (2, 3)
 (6, 19)
 (6, 23)
 (6, 24)
 (14, 15)
 (17, 18)
 (19, 23)
 (19, 24)
 (21, 26)
 (22, 25)
 (23, 24)

In [30]:
first(transformation(X_train_solar), 5) #all transformations

,X1,Sqrt1,Abs1,Log1,X2,Sqrt2,Abs2,Log2,X3
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,32.4,1049.76,5.6921,3.50856,29.7,882.09,5.44977,3.42426,24.0
2,32.5,1056.25,5.70088,3.51155,32.5,1056.25,5.70088,3.51155,24.1
3,31.6,998.56,5.62139,3.48431,28.3,800.89,5.31977,3.37759,24.2
4,31.6,998.56,5.62139,3.48431,26.8,718.24,5.17687,3.32504,25.1
5,31.8,1011.24,5.63915,3.49043,27.2,739.84,5.21536,3.33932,25.2


In [57]:
#normalize data
X_train_solar_norm=mapslices(X_train_solar -> X_train_solar / norm(X_train_solar), X_train_solar, dims=1)
y_train_solar_norm=mapslices(y_train_solar -> y_train_solar / norm(y_train_solar), y_train_solar, dims=1)
X_valid_solar_norm=mapslices(X_valid_solar -> X_valid_solar / norm(X_valid_solar), X_valid_solar, dims=1)
y_valid_solar_norm=mapslices(y_valid_solar -> y_valid_solar / norm(y_valid_solar), y_valid_solar, dims=1);

In [83]:
beta_opt=holistic_regression(X_train_solar_norm, y_train_solar_norm, 0.2, 0.5, 50, 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


104-element Vector{Float64}:
 0.0
 0.0
 0.0009396879490121381
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [84]:
#return indices of selected features
selected_features=findall(x->x>0, beta_opt)

3-element Vector{Int64}:
  3
 43
 55

In [85]:
#what is the name of the selected features
names(transformation(X_train_solar))[selected_features]

3-element Vector{String}:
 "Abs1"
 "Abs11"
 "Abs14"

* Absolute squared value of temperature
* Absolute squared value of sealevelpressure
* Absolute squared value of solarirradiation

In [58]:
#report mse
X_valid_solar_trans=Matrix(transformation(X_valid_solar));
mse(X_valid_solar_trans, y_valid_solar, beta_opt)
#should i normalize it to compute mse?
X_valid_solar_norm_trans=Matrix(transformation(X_valid_solar_norm))
mse(X_valid_solar_norm_trans, y_valid_solar_norm, beta_opt)

5.749173749642126e-5

                DO CROSS VALIDATION TO SEE WHICH HYPERPARAMETERS ARE BETTER FOR HOLISTIC REGRESSION!

### WIND ENERGY OUTPUT

In [60]:
beta_lasso=lasso(X_train_wind,y_train_wind,0.25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


26-element Vector{Float64}:
   -301.84854437880716
    -13.878654022084413
    285.76408936346456
   -169.5638772461184
   1966.477390343377
   1398.6358008255831
   -582.6406033857658
   -154.61732330247344
    518.5437405897798
      5.67971621341388
   -121.20450008851935
    -14.587541729006283
    148.87811658371598
     -6.613065161257459
    128.61541834002932
    354.98350834484785
   -103.27849515246002
    162.24750297155276
    247.14959700890893
      5.26182812062609e-11
   -181.83304268421628
     -2.7204296202842075e-11
 139498.7346158648
    847.1709151858244
     -2.7204296202842075e-11
   -181.83304268421628

In [61]:
mse(X_valid_wind, y_valid_wind, beta_lasso)

1.170538230487726e7

In [62]:
correlation(X_train_wind,0.6) #correlated features

13-element Vector{Any}:
 (1, 2)
 (1, 3)
 (2, 3)
 (6, 19)
 (6, 23)
 (6, 24)
 (14, 15)
 (17, 18)
 (19, 23)
 (19, 24)
 (21, 26)
 (22, 25)
 (23, 24)

In [63]:
#normalize data
X_train_wind_norm=mapslices(X_train_wind -> X_train_wind / norm(X_train_wind), X_train_wind, dims=1)
y_train_wind_norm=mapslices(y_train_wind -> y_train_wind / norm(y_train_wind), y_train_wind, dims=1)
X_valid_wind_norm=mapslices(X_valid_wind -> X_valid_wind / norm(X_valid_wind), X_valid_wind, dims=1)
y_valid_wind_norm=mapslices(y_valid_wind -> y_valid_wind / norm(y_valid_wind), y_valid_wind, dims=1);

In [74]:
#holistic regression
beta_opt=holistic_regression(X_train_wind_norm, y_train_wind_norm, 0.05, 0.45, 50, 10)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


104-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.030786713180086894
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.004607147618597973
  0.0
  0.0
  0.0
 -0.002267068103363347
  0.0
  0.0
  0.0
  0.0
  0.0

In [76]:
#return indices of selected features
selected_features=findall(x->x>0, beta_opt)

3-element Vector{Int64}:
 35
 39
 95

In [80]:
#what is the name of the features?
names(transformation(X_train_wind))[selected_features]

3-element Vector{String}:
 "Abs9"
 "Abs10"
 "Abs24"

* Absolute value of windspeed
* Absolute value of winddirection
* Absolute value of precipitation type rain


In [77]:
#mse
X_valid_wind_norm_trans=Matrix(transformation(X_valid_wind_norm))
mse(X_valid_wind_norm_trans, y_valid_wind_norm, beta_opt)

0.00046643056249818436